In [1]:
import pandas as pd

In [2]:
data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype({'requester_id':'Int64', 'accepter_id':'Int64', 'accept_date':'datetime64[ns]'})
request_accepted

,requester_id,accepter_id,accept_date
0,1,2,2016-06-03
1,1,3,2016-06-08
2,2,3,2016-06-08
3,3,4,2016-06-09


In [ ]:
req = request_accepted['requester_id']
acc = request_accepted['accepter_id']
req_acc = pd.concat([req, acc])
req_acc.value_counts()
cnt = req_acc.value_counts()
cnt[cnt == cnt.max()]
pd.DataFrame({'id': cnt[cnt == cnt.max()].index, 'num': cnt[cnt == cnt.max()].values})

3    3
1    2
2    2
4    1
Name: count, dtype: Int64

value_counts() 사용하는 방법

In [44]:
request_accepted.groupby('requester_id')['accepter_id'].count()

requester_id
1    2
2    1
3    1
Name: accepter_id, dtype: Int64

In [49]:
rv = request_accepted['requester_id'].value_counts()
av = request_accepted['accepter_id'].value_counts()
ans = rv.add(av, fill_value=0)
print((ans.idxmax()))
print((ans.max()))

3
3


## 607. Sales Person

Write a solution to find the names of all the salespersons who did not have any orders related to the company with the name "RED".

Return the result table in any order.

The result format is in the following example.

In [63]:
data = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
sales_person = pd.DataFrame(data, columns=['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']).astype({'sales_id':'Int64', 'name':'object', 'salary':'Int64', 'commission_rate':'Int64', 'hire_date':'datetime64[ns]'})
data = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
company = pd.DataFrame(data, columns=['com_id', 'name', 'city']).astype({'com_id':'Int64', 'name':'object', 'city':'object'})
data = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'com_id', 'sales_id', 'amount']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'com_id':'Int64', 'sales_id':'Int64', 'amount':'Int64'})
display(sales_person)
display(company)
display(orders)

,sales_id,name,salary,commission_rate,hire_date
0,1,John,100000,6,2006-04-01
1,2,Amy,12000,5,2010-05-01
2,3,Mark,65000,12,2008-12-25
3,4,Pam,25000,25,2005-01-01
4,5,Alex,5000,10,2007-02-03


,com_id,name,city
0,1,RED,Boston
1,2,ORANGE,New York
2,3,YELLOW,Boston
3,4,GREEN,Austin


,order_id,order_date,com_id,sales_id,amount
0,1,2014-01-01,3,4,10000
1,2,2014-02-01,4,5,5000
2,3,2014-03-01,1,1,50000
3,4,2014-04-01,1,4,25000


In [81]:
red_id = company[company['name'] == 'RED']['com_id']
red_order_id = orders[orders['com_id'].isin(red_id)]['sales_id']
pd.DataFrame(sales_person[~sales_person['sales_id'].isin(red_order_id)]['name'])

,name
1,Amy
2,Mark
4,Alex


### **merge 연습하기**

In [ ]:
def sales_person(sales_person: pd.DataFrame, company: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    df1 = pd.merge(orders,company,on='com_id', how='left')[['sales_id','name']]
    df=  pd.merge(sales_person,df1, on='sales_id',how='left').rename(columns={'name_x':'name','name_y':'company_name'})[['name','company_name']]
    df = df[df['company_name']=="RED"][['name']]
    df3 = sales_person[~sales_person['name'].isin(df['name'])][['name']]
    return df3

2    1
3    4
Name: sales_id, dtype: Int64

In [85]:
df1 = pd.merge(orders,company,on='com_id', how='left')[['sales_id','name']]
pd.merge(sales_person,df1, on='sales_id',how='left')

,sales_id,name_x,salary,commission_rate,hire_date,name_y
0,1,John,100000,6,2006-04-01,RED
1,2,Amy,12000,5,2010-05-01,NaN
2,3,Mark,65000,12,2008-12-25,NaN
3,4,Pam,25000,25,2005-01-01,YELLOW
4,4,Pam,25000,25,2005-01-01,RED
5,5,Alex,5000,10,2007-02-03,GREEN


## 608. Tree Node

Each node in the tree can be one of three types:

- "Leaf": if the node is a leaf node.
- "Root": if the node is the root of the tree.
- "Inner": If the node is neither a leaf node nor a root node.

Write a solution to report the type of each node in the tree.

Return the result table in any order.

The result format is in the following example.

In [12]:
data = [[1, None], [2, 1], [3, 1], [4, 2], [5, 2]]
tree = pd.DataFrame(data, columns=['id', 'p_id']).astype({'id':'Int64', 'p_id':'Int64'})
display(tree)

,id,p_id
0,1,<NA>
1,2,1
2,3,1
3,4,2
4,5,2


In [3]:
start = tree['id'].unique()
end = tree['p_id'].unique()
leaf = tree[~tree['id'].isin(end)]['id']
root = tree[~tree['id'].isin(tree.dropna()['id'])]['id']
inner = tree[~tree['id'].isin(pd.concat([root, leaf]))]['id']
pd.DataFrame({'id': pd.concat([root, inner, leaf]), 'type': ['Root']*len(root) + ['Inner']*len(inner) + ['Leaf']*len(leaf)})

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


- id 중에 p_id에 없으면 leaf 노드
- p_id null인 id는 root 노드
- 나머지는 inner 노드

In [ ]:
def tree_node(tree: pd.DataFrame) -> pd.DataFrame:
    if tree.empty:
        return pd.DataFrame(columns=['id', 'type'])
    
    node_id = tree['id']
    parent_id = tree['p_id'].dropna().unique()          # tree['p_id] 만 했을 땐 결과가 완전히 다르다.
    
    def get_type(tree):
        if pd.isna(tree['p_id']):
            return 'Root'
        elif tree['id'] in parent_id:
            return 'Inner'
        else:
            return 'Leaf'
    
    tree['type'] = tree.apply(get_type, axis=1)

    return tree[['id', 'type']]
tree_node(tree)

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


참고 답안

In [27]:
import warnings
# def tree_node(tree: pd.DataFrame) -> pd.DataFrame:
#     parents = set(tree['p_id'].dropna())
    
#     is_root = tree['p_id'].isna()
#     is_inner = tree['id'].isin(parents) & ~is_root
#     is_leaf = ~is_root & ~is_inner

#     tree['type'] = ''

def tree_node(tree: pd.DataFrame) -> pd.DataFrame:
    tree['type'] = 'Leaf'
    tree['type'].loc[tree['id'].isin(tree['p_id'])] = 'Inner'
    tree['type'].loc[tree['p_id'].isna()] = 'Root'

    return tree[['id','type']]

display(tree_node(tree))

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_20876\3778313973.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  tree['type'].loc[tree['id'].isin(tree['p_id'])] = 'Inner'
C:\Users\SSAFY\AppData\Local\Temp\ipykernel_20876\3778313973.py:13: S

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


## 610. 

Report for every three line segments whether they can form a triangle.

Return the result table in any order.

The result format is in the following example.

In [73]:
data = [[13, 15, 30], [10, 20, 15]]
triangle = pd.DataFrame(data, columns=['x', 'y', 'z']).astype({'x':'Int64', 'y':'Int64', 'z':'Int64'})
triangle.T

,0,1
x,13,10
y,15,20
z,30,15


In [69]:
def triangle_judgement(triangle: pd.DataFrame) -> pd.DataFrame:
    def check_triangle(row):
        if sum(row) - 2 * max(row) > 0:
            return 'Yes'
        else:
            return 'No'
    
    triangle['triangle'] = triangle.apply(check_triangle, axis=1)
    return triangle
triangle_judgement(triangle)

,x,y,z,triangle
0,13,15,30,No
1,10,20,15,Yes


In [74]:
import pandas as pd

def triangle_judgement(triangle: pd.DataFrame) -> pd.DataFrame:
    
    def check(triangle):
        if int(sum(triangle)) - int(2*max(triangle)) > 0:
            return 'Yes'
        else:
            return 'No'
    
    triangle['triangle'] = triangle.apply(check, axis=1)
    return triangle
triangle_judgement(triangle)

,x,y,z,triangle
0,13,15,30,No
1,10,20,15,Yes


## Biggest Single Number

A single number is a number that appeared only once in the MyNumbers table.

Find the largest single number. If there is no single number, report null.

The result format is in the following example.

In [53]:
data = [[8], [8], [3], [3], [1], [4], [5], [6]]
my_numbers = pd.DataFrame(data, columns=['num']).astype({'num':'Int64'})

In [54]:
data = [[8], [8], [3], [3]]
my_numbers = pd.DataFrame(data, columns=['num']).astype({'num':'Int64'})
# not considered null yet
cnt = my_numbers.value_counts()
ans = cnt[cnt == 1].index.max()
if pd.notna(ans):
    display(pd.DataFrame({'num': ans}))
else:
    display(pd.DataFrame({'num': [pd.NA]}))

,num
0,<NA>


In [52]:
my_numbers['num'].drop_duplicates(keep='first')

0    8
2    3
4    1
5    4
6    5
7    6
Name: num, dtype: Int64

## 620. Not Boring Movies

Write a solution to report the movies with an odd-numbered ID and a description that is not "boring".

Return the result table ordered by rating in descending order.

The result format is in the following example.

In [2]:
import pandas as pd

In [3]:
data = [[1, 'War', 'great 3D', 8.9], [2, 'Science', 'fiction', 8.5], [3, 'irish', 'boring', 6.2], [4, 'Ice song', 'Fantacy', 8.6], [5, 'House card', 'Interesting', 9.1]]
cinema = pd.DataFrame(data, columns=['id', 'movie', 'description', 'rating']).astype({'id':'Int64', 'movie':'object', 'description':'object', 'rating':'Float64'})
cinema

,id,movie,description,rating
0,1,War,great 3D,8.9
1,2,Science,fiction,8.5
2,3,irish,boring,6.2
3,4,Ice song,Fantacy,8.6
4,5,House card,Interesting,9.1


In [7]:
# id 홀수인 것만 filtering
cinema_odd = cinema[cinema['id'] % 2 == 1]
# description에서 boring인 것 제외
cinema_good = cinema_odd[cinema_odd['description'] != 'boring']
# rating 기준으로 내림차순
ans = cinema_good.sort_values('rating', ascending=False)
ans

,id,movie,description,rating
4,5,House card,Interesting,9.1
0,1,War,great 3D,8.9


## 626. Exchange Seats

Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.

Return the result table ordered by id in ascending order.

The result format is in the following example.

In [43]:
data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id':'Int64', 'student':'object'})
seat

,id,student
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


In [ ]:
odd = seat[seat['id'] % 2 == 1]
even = seat[seat['id'] % 2 == 0]
even['id'] = even['id'] - 1
if seat['id'].max() % 2 == 0:
    odd['id'] = odd['id'] + 1
else:
    odd['id'].iloc[:-1] = odd['id'].iloc[:-1] + 1
ans = pd.concat([odd, even])
ans = ans.sort_values('id', ascending=True)
ans

C:\Users\User\AppData\Local\Temp\ipykernel_17908\877344839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  even['id'] = even['id'] - 1
C:\Users\User\AppData\Local\Temp\ipykernel_17908\877344839.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = 

,id,student
0,2,Abbot
2,4,Emerson
4,5,Jeames
1,1,Doris
3,3,Green


다른 풀이

In [48]:
def exchange_seats(seat: pd.DataFrame) -> pd.DataFrame:
    n= len(seat)
    
    def swap(id):
        if id % 2 == 0:
            return id - 1
        elif id % 2 == 1 and id < n:
            return id + 1
        elif id % 2 == 1 and id == n:
            return id
    
    seat['id'] = seat['id'].apply(swap)

    return seat.sort_values('id')
exchange_seats(seat)

,id,student
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames
